In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/AIFFEL_quest_rs/Exploration/Quest01/
!ls

In [1]:
!pwd

/home/jovyan/work/workplace/AIFFEL_quest_rs/Exploration/Quest01


In [ ]:
!git pull origin main

From https://github.com/nagujean/AIFFEL_quest_rs
 * branch            main       -> FETCH_HEAD
Auto packing the repository in background for optimum performance.
See "git help gc" for manual housekeeping.
and remove .git/gc.log
Automatic cleanup will not be performed until the file is removed.


Already up to date.


In [24]:
!git add .

In [2]:
  !git config --global user.email "nagujean@gmail.com"
  !git config --global user.name "nagujean"

In [25]:
!git commit -m "commit"

[main c33f504] commit
 2 files changed, 1332 insertions(+), 24 deletions(-)
 create mode 100644 Exploration/Quest01/.ipynb_checkpoints/pytorch_1-checkpoint.ipynb


In [26]:
!git push origin main

Username for 'https://github.com': ^C


In [23]:
!rm -rf "/home/jovyan/work/workplace/AIFFEL_quest_rs/Exploration/Quest01/C:/Users/KHI/Desktop/mldata"

In [28]:
import torch
import os
from torch import nn, optim
from torch.nn import functional as F
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
from torch.utils.data import random_split

In [29]:
target_dataset_dir = os.path.abspath(os.path.join(os.getcwd(), "../../../datasets/mldata")) 


transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])


train_dataset = datasets.CIFAR10(root=target_dataset_dir, train=True, download=True, transform=transform)
val_dataset = datasets.CIFAR10(root=target_dataset_dir, train=False, download=True, transform=transform)

val_size = len(val_dataset)
print(val_size)
test_size = val_size // 2
print(test_size)
new_val_size = val_size - test_size
print(new_val_size)

val_dataset, test_dataset = random_split(val_dataset, [new_val_size, test_size], generator=torch.Generator().manual_seed(2025))

train_dataloader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=32, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=32, shuffle=True)

10000
5000
5000


In [30]:
print(f"현재 파이썬 코드가 사용하는 데이터셋 저장 경로: {target_dataset_dir}")

현재 파이썬 코드가 사용하는 데이터셋 저장 경로: /home/jovyan/work/workplace/datasets/mldata


In [31]:
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break

Shape of X [N, C, H, W]: torch.Size([32, 3, 32, 32])
Shape of y: torch.Size([32]) torch.int64


In [32]:
# 학습에 사용할 CPU나 GPU, MPS 장치를 얻습니다.
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

# 모델을 정의합니다.
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(3*32*32, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10)
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

model = NeuralNetwork().to(device)
print(model)

Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=3072, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


In [33]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [34]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # 예측 오류 계산
        pred = model(X)
        loss = loss_fn(pred, y)

        # 역전파
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [35]:
def val(dataloader, model, loss_fn):
    model.eval()
    val_loss = 0
    correct = 0
    total = 0

    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            val_loss += loss_fn(pred, y).item()

            correct += (pred.argmax(1) == y).sum().item()
            total += y.size(0)

    avg_loss = val_loss / len(dataloader)
    accuracy = correct / total * 100
    print(f"Validation loss: {avg_loss:.4f}, Accuracy: {accuracy:.2f}%\n")

In [36]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [37]:
epochs = 5
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    val(val_dataloader, model, loss_fn)
    test(test_dataloader, model, loss_fn)
print("Done!")



Epoch 1
-------------------------------
loss: 2.313846  [   32/50000]
loss: 2.299055  [ 3232/50000]
loss: 2.258565  [ 6432/50000]
loss: 2.218471  [ 9632/50000]
loss: 2.170770  [12832/50000]
loss: 2.195712  [16032/50000]
loss: 2.164032  [19232/50000]
loss: 2.085094  [22432/50000]
loss: 2.117492  [25632/50000]
loss: 2.123585  [28832/50000]
loss: 2.304708  [32032/50000]
loss: 2.136848  [35232/50000]
loss: 1.971008  [38432/50000]
loss: 2.043857  [41632/50000]
loss: 1.977215  [44832/50000]
loss: 1.932513  [48032/50000]
Validation loss: 1.9837, Accuracy: 30.84%

Test Error: 
 Accuracy: 31.9%, Avg loss: 1.979762 

Epoch 2
-------------------------------
loss: 1.936919  [   32/50000]
loss: 2.046080  [ 3232/50000]
loss: 1.965659  [ 6432/50000]
loss: 1.832719  [ 9632/50000]
loss: 2.050475  [12832/50000]
loss: 1.910905  [16032/50000]
loss: 1.694771  [19232/50000]
loss: 1.880256  [22432/50000]
loss: 1.800857  [25632/50000]
loss: 2.082801  [28832/50000]
loss: 1.861462  [32032/50000]
loss: 2.060397 

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

In [ ]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

In [40]:
classes = [
    "tench",
    "English springer",
    "cassette player",
    "chain saw",
    "church",
    "French horn",
    "garbage truck",
    "gas pump",
    "golf ball",
    "parachute",
]




model.eval()
x, y = test_dataset[0][0], test_dataset[0][1]
with torch.no_grad():
    x = x.unsqueeze(0).to(device)
    pred = model(x)
    predicted = classes[pred[0].argmax(0)]
    actual = classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "garbage truck", Actual: "French horn"
